In [506]:
import Bens_forecasting_utils as fc
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
%matplotlib ipympl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from pandas.tseries.offsets import MonthEnd
plt.rcParams['figure.figsize'] = [10, 5]
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
pd.set_option('display.float_format', '{:,.0f}'.format)


pd.options.display.max_rows = None

In [507]:
parameters = dict(
                sheet_id = '1cbW6p8rdMhq9XD5WL-QJNO1UBRQRTcKQBoh6g1QfEv8',
                copied_marketing_file = 'actual_budget_2025!A1:Y9',
                tnc_mapping = 'actual_budget_2025!A13:F20')

#read the correct mau table under first tab in marketing file
marketing_df = fc.import_gsheet_to_df(parameters["sheet_id"], parameters["copied_marketing_file"]) 
marketing_df.head()


,Cum. Net MAU,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,Jul 24,Aug 24,Sep 24,...,Mar 25,Apr 25,May 25,Jun 25,Jul 25,Aug 25,Sep 25,Oct 25,Nov 25,Dec 25
1,Germany,"938,923","948,289","962,084","976,868","985,266","999,052","1,015,667","1,025,103","1,048,425",...,"1,177,719","1,200,143","1,224,672","1,249,421","1,275,007","1,302,911","1,336,537","1,371,818","1,403,034","1,429,577"
2,Austria,"92,804","93,592","94,898","96,202","97,492","98,712","100,853","101,771","104,294",...,"118,936","121,912","124,980","127,844","130,703","133,500","136,863","139,999","142,582","144,815"
3,France,"402,334","402,655","403,002","405,338","405,793","406,600","413,274","418,117","428,581",...,"484,404","496,014","507,770","518,351","530,132","543,210","558,813","576,143","592,228","603,553"
4,Italy,"221,169","218,012","216,048","213,864","211,890","210,061","211,772","211,710","212,814",...,"210,992","210,114","209,220","211,361","213,989","218,318","223,955","227,275","230,125","231,764"
5,Spain,"242,213","247,998","254,786","257,942","262,081","269,578","278,198","282,133","285,685",...,"332,118","342,416","353,348","362,847","373,065","383,571","397,672","413,827","428,475","440,860"


### melt the language mapping table

In [508]:
tnc_df = fc.import_gsheet_to_df(parameters["sheet_id"], parameters["tnc_mapping"]) 
tnc_df.head()

,TnC Country,de,en,es,fr,it
1,AUT,80.01685962280600%,18.19300742336240%,1.08882392912548%,0.21923920150775%,0.48206982319833%
2,DEU,64.66037909412580%,30.27668427472610%,3.34889159954434%,0.83728590582377%,0.87675912578001%
3,ESP,0.80560083284449%,18.12758380033990%,78.92836192565150%,1.30282661670836%,0.83562682445576%
4,FRA,0.26591651752479%,7.89476767848787%,1.34442516645171%,90.21756876996420%,0.27732186757139%
5,GrE,1.84452639284597%,84.93585227292250%,3.65124807110036%,8.82121463111191%,0.74715863201922%


In [509]:
tnc_melted = pd.melt(tnc_df, id_vars=["TnC Country"], var_name="Language", value_name="Percentage")

tnc_melted["Country_Lang"] = tnc_melted["TnC Country"] + ":" + tnc_melted["Language"]
tnc_melted = tnc_melted[["Country_Lang", "Percentage"]]
tnc_melted.head()

,Country_Lang,Percentage
0,AUT:de,80.01685962280600%
1,DEU:de,64.66037909412580%
2,ESP:de,0.80560083284449%
3,FRA:de,0.26591651752479%
4,GrE:de,1.84452639284597%


In [510]:
 tnc_df.columns.tolist()

['TnC Country', 'de', 'en', 'es', 'fr', 'it']

In [511]:
# perform table transformations
marketing_df = marketing_df.T  
marketing_df = marketing_df.reset_index()
marketing_df.columns = ["Cum. Net MAU"] + list(marketing_df.iloc[0, 1:])  
marketing_df = marketing_df[1:].reset_index(drop=True)
marketing_df.rename(columns={"Cum. Net MAU": "Months"}, inplace=True)
marketing_df.head()

,Months,Germany,Austria,France,Italy,Spain,Greater Europe,Non-Euro,Total
0,Jan 24,"938,923","92,804","402,334","221,169","242,213","193,827","27,107","2,118,377"
1,Feb 24,"948,289","93,592","402,655","218,012","247,998","193,671","27,310","2,131,527"
2,Mar 24,"962,084","94,898","403,002","216,048","254,786","197,417","27,305","2,155,540"
3,Apr 24,"976,868","96,202","405,338","213,864","257,942","196,750","27,429","2,174,393"
4,May 24,"985,266","97,492","405,793","211,890","262,081","197,474","27,604","2,187,600"


### Mapp by % language split 

In [512]:
def apply_language(marketing_df, tnc_melted):
    # Clean up column names
    tnc_melted.columns = tnc_melted.columns.str.strip()
    tnc_melted['Percentage'] = tnc_melted['Percentage'].astype(str)
    tnc_melted['Percentage'] = tnc_melted['Percentage'].str.rstrip('%')
    tnc_melted['Percentage'] = pd.to_numeric(tnc_melted['Percentage'], errors='coerce') / 100

    tnc_melted['Country'] = tnc_melted['Country_Lang'].str.split(':').str[0]  
    tnc_melted['Language'] = tnc_melted['Country_Lang'].str.split(':').str[-1] 

    # mapping country 
    country_columns = {
        'DEU': 'Germany',
        'AUT': 'Austria',
        'FRA': 'France',
        'ITA': 'Italy',
        'ESP': 'Spain',
        'GrE': 'Greater Europe',
        'NEuro': 'Non-Euro'}

    expanded_rows = []

    for _, row in marketing_df.iterrows():
        month = row['Months']  

        for country_code, column in country_columns.items(): 
            # Convert string with commas to float with full precision
            if pd.isna(row[column]):
                continue
                
            total_value = float(str(row[column]).replace(',', ''))
            
            country_languages = tnc_melted[tnc_melted['Country'] == country_code]  

            for _, lang_row in country_languages.iterrows():
                language = lang_row['Language']
                percentage = lang_row['Percentage']

                language_value = total_value * percentage 

                new_row = {
                    'Calendar Week': month,
                    'Country Name': column,
                    'Country': country_code,
                    'Language': language,
                    'Total Country MAU': total_value,
                    'Language Percentage': percentage,
                    'MAU': language_value}

                expanded_rows.append(new_row)

    expanded_df = pd.DataFrame(expanded_rows)
    aggregated_df = expanded_df.groupby(['Calendar Week', 'Language'], as_index=False)['MAU'].sum()
    aggregated_df['MAU'] = aggregated_df['MAU'].round(0)
    
    return aggregated_df

In [513]:
mapped_df = apply_language(marketing_df, tnc_melted)
mapped_df.head()

,Calendar Week,Language,MAU
0,Apr 24,de,"724,162"
1,Apr 24,en,"593,475"
2,Apr 24,es,"254,293"
3,Apr 24,fr,"397,507"
4,Apr 24,it,"204,956"


#### Sort df as per consecutive months and calculate MoM % change

#### Calculate % MoM for each monthly:language pairing

In [514]:
mapped_df['MAU'] = mapped_df['MAU'].astype(str).str.replace(',', '').astype(float)

def month_to_num(month_abbr):
    month_dict = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    return month_dict.get(month_abbr.title(), 0)

# Extract month and year
mapped_df[['month', 'year']] = mapped_df['Calendar Week'].str.extract(r'(\w+)\s*(\d{2})')
mapped_df['year'] = mapped_df['year'].astype(int)
mapped_df['month_num'] = mapped_df['month'].apply(month_to_num)

# Sort by language, year, and month
sorted_df = mapped_df.sort_values(by=['Language', 'year', 'month_num'])

# MoM Growth Calculation (Now we calculate MoM Growth after sorting)
sorted_df['MoM Growth (%)'] = sorted_df.groupby('Language')['MAU'].pct_change() * 100

# Format the MoM Growth as desired (8 decimal places)
sorted_df['MoM Growth (%)'] = sorted_df['MoM Growth (%)'].apply(lambda x: f'{x:.8f}' if not pd.isna(x) else 'NaN')

# Drop temporary columns used for sorting
sorted_df = sorted_df.drop(columns=['month_num'])

# Reset index for clean output
sorted_df = sorted_df.reset_index(drop=True)
sorted_df.head(10)


,Calendar Week,Language,MAU,month,year,MoM Growth (%)
0,Jan 24,de,"696,736",Jan,24,NaN
1,Feb 24,de,"703,474",Feb,24,0.96708079
2,Mar 24,de,"713,538",Mar,24,1.43061435
3,Apr 24,de,"724,162",Apr,24,1.48891860
4,May 24,de,"730,687",May,24,0.90104148
5,Jun 24,de,"740,731",Jun,24,1.37459678
6,Jul 24,de,"753,518",Jul,24,1.72626770
7,Aug 24,de,"760,393",Aug,24,0.91238696
8,Sep 24,de,"777,574",Sep,24,2.25948950
9,Oct 24,de,"792,642",Oct,24,1.93782200


In [515]:
sorted_df['MoM Growth (%)'] = pd.to_numeric(sorted_df['MoM Growth (%)'], errors='coerce')

In [516]:
sorted_df.columns

Index(['Calendar Week', 'Language', 'MAU', 'month', 'year', 'MoM Growth (%)'], dtype='object')

### Add weeks and leave it NaN except last week per each month

In [517]:
import calendar

def create_weekly_data(sorted_df):
    # Ensure MAU is numeric
    sorted_df['MAU'] = sorted_df['MAU'].astype(float)

    # Function to convert month abbreviation to number
    def month_to_num(month_abbr):
        month_dict = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
        
        return month_dict.get(month_abbr.title(), 0)

    # Extract month number
    sorted_df['month_num'] = sorted_df['month'].apply(month_to_num)

    # Sort DataFrame properly
    sorted_df = sorted_df.sort_values(by=['Language', 'year', 'month_num'])

    # Initialize list to store weekly data
    week_data = []

    # Loop through each language
    for lang in sorted_df['Language'].unique():
        lang_data = sorted_df[sorted_df['Language'] == lang]

        for _, row in lang_data.iterrows():
            year = row['year']
            month = row['month_num']
            last_value = row['MAU']  # Monthly absolute value

            # Get all Mondays of the month using `calendar` module
            cal = calendar.monthcalendar(year, month)
            mondays = [day for week in cal for day in [week[0]] if day != 0]  
            
            # Convert to actual dates
            monday_dates = [pd.Timestamp(year=year, month=month, day=day) for day in mondays]

            # Identify the last Monday of the month
            last_monday = monday_dates[-1]

            # Append weekly data
            for date in monday_dates:
                week_data.append({
                    'Week Starting': date.strftime('%d/%m/%Y'),
                    'Language/Market/Iban': lang,
                    'Type': 'abs' if date == last_monday else 'interpolated',
                    'MAU': last_value if date == last_monday else np.nan, })

    # Create weekly DataFrame
    weekly_df = pd.DataFrame(week_data)
    
    return weekly_df

# Example Usage:
weekly_df = create_weekly_data(sorted_df)
weekly_df = weekly_df[4:]
weekly_df.head(15)


,Week Starting,Language/Market/Iban,Type,MAU
4,29/01/0024,de,abs,"696,736"
5,05/02/0024,de,interpolated,NaN
6,12/02/0024,de,interpolated,NaN
7,19/02/0024,de,interpolated,NaN
8,26/02/0024,de,abs,"703,474"
9,04/03/0024,de,interpolated,NaN
10,11/03/0024,de,interpolated,NaN
11,18/03/0024,de,interpolated,NaN
12,25/03/0024,de,abs,"713,538"
13,01/04/0024,de,interpolated,NaN


#### Calcualte weekly % after having input weeks starting from Mondays in each month

In [518]:
import numpy as np
import pandas as pd
import calendar

def create_weekly_data(sorted_df):
    # Ensure MAU is numeric
    sorted_df['MAU'] = sorted_df['MAU'].astype(str).str.replace(',', '').astype(float)

    # Ensure year is an integer
    sorted_df['year'] = sorted_df['year'].astype(int)

    # Function to convert month abbreviation to number
    def month_to_num(month_abbr):
        month_dict = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
        return month_dict.get(month_abbr.title(), 0)

    # Extract month number
    sorted_df['month_num'] = sorted_df['month'].apply(month_to_num)

    # Sort DataFrame properly
    sorted_df = sorted_df.sort_values(by=['Language', 'year', 'month_num'])

    # Compute MoM Growth
    sorted_df['MoM Growth (%)'] = sorted_df.groupby('Language')['MAU'].pct_change() * 100

    # List to store weekly data
    week_data = []

    # Iterate over each language separately
    for lang in sorted_df['Language'].unique():
        lang_data = sorted_df[sorted_df['Language'] == lang]

        for _, row in lang_data.iterrows():
            year = row['year']
            month = row['month_num']
            last_value = row['MAU']  # Monthly absolute value
            mom_growth = row['MoM Growth (%)']

            # Calculate weekly growth rate for this month based on the MoM Growth
            if not np.isnan(mom_growth):
                weekly_growth = (1 + mom_growth / 100) ** (1 / 4) - 1
            else:
                weekly_growth = 0  # If no MoM Growth available, assume 0% growth

            # Get all Mondays of the month using `calendar` module
            cal = calendar.monthcalendar(year, month)
            mondays = [week[0] for week in cal if week[0] != 0]  

            # Convert to actual dates
            week_starting_dates = [pd.Timestamp(year, month, day).strftime('%d/%m/%Y') for day in mondays]

            num_weeks = len(week_starting_dates)  # Handle months with 4 or 5 weeks dynamically
            
            # Initialize weekly values with NaNs
            weekly_values = [np.nan] * num_weeks

            # Set last week’s value to the monthly absolute value
            weekly_values[-1] = last_value

            # Back-interpolate missing values
            for i in range(num_weeks - 2, -1, -1):  # Start from second-to-last week and go backwards
                weekly_values[i] = weekly_values[i + 1] / (1 + weekly_growth)  # Correct backward interpolation

            # Append weekly data
            for i in range(num_weeks):
                week_data.append({
                    'Week Starting': week_starting_dates[i],
                    'Language/Market/Iban': lang,
                    'Type': 'abs' if i == num_weeks - 1 else 'interpolated',
                    'MAU': round(weekly_values[i]),
                    'weekly % Change': f"{weekly_growth * 100:.2f}%" if i != num_weeks - 1 else ''})

    # Create weekly DataFrame
    weekly_df = pd.DataFrame(week_data)

    return weekly_df

# Example Usage:
weekly_df = create_weekly_data(sorted_df)
weekly_df= weekly_df[4:]
weekly_df.head(15)


,Week Starting,Language/Market/Iban,Type,MAU,weekly % Change
4,29/01/0024,de,abs,696736,
5,05/02/0024,de,interpolated,698414,0.24%
6,12/02/0024,de,interpolated,700097,0.24%
7,19/02/0024,de,interpolated,701783,0.24%
8,26/02/0024,de,abs,703474,
9,04/03/0024,de,interpolated,705977,0.36%
10,11/03/0024,de,interpolated,708488,0.36%
11,18/03/0024,de,interpolated,711009,0.36%
12,25/03/0024,de,abs,713538,
13,01/04/0024,de,interpolated,713538,0.37%


In [519]:
weekly_df['Week Starting'] = weekly_df['Week Starting'].apply(lambda x: x if isinstance(x, str) else str(x)) 
weekly_df['Week Starting'] = weekly_df['Week Starting'].apply(lambda x: x.replace('0024', '2024').replace('0025', '2025') if '0024' in x or '0025' in x else x)
weekly_df['Week Starting'] = pd.to_datetime(weekly_df['Week Starting'], format='%d/%m/%Y', errors='coerce')

# weekly_df[['Week Starting']].head()



In [520]:
weekly_df = weekly_df.drop(columns=['weekly % Change'], errors='ignore')

weekly_df['% Change'] = weekly_df.groupby(['Language/Market/Iban'])['MAU'].pct_change() * 100

weekly_df['% Change'] = weekly_df['% Change'].apply(lambda x: f"{x:.2f}" if pd.notnull(x) else '')

def get_calendar_week(date):
    date = pd.to_datetime(date)
    iso_year, iso_week, iso_weekday = date.isocalendar()
    return f"{iso_year}-w{iso_week:02d}"

weekly_df['Calendar Week'] = weekly_df['Week Starting'].apply(get_calendar_week)

weekly_df.loc[weekly_df['Language/Market/Iban'] != weekly_df['Language/Market/Iban'].shift(), '% Change'] = ''
weekly_df.head(15)


,Week Starting,Language/Market/Iban,Type,MAU,% Change,Calendar Week
4,2024-01-29,de,abs,696736,,2024-w05
5,2024-02-05,de,interpolated,698414,0.24,2024-w06
6,2024-02-12,de,interpolated,700097,0.24,2024-w07
7,2024-02-19,de,interpolated,701783,0.24,2024-w08
8,2024-02-26,de,abs,703474,0.24,2024-w09
9,2024-03-04,de,interpolated,705977,0.36,2024-w10
10,2024-03-11,de,interpolated,708488,0.36,2024-w11
11,2024-03-18,de,interpolated,711009,0.36,2024-w12
12,2024-03-25,de,abs,713538,0.36,2024-w13
13,2024-04-01,de,interpolated,713538,0.00,2024-w14


In [521]:
# weekly_df

In [553]:
import numpy as np
import pandas as pd

# Function to apply smoothing only to 'interpolated' rows with identical 'MAU' values
def smooth_interpolate_mau(df, smoothness_factor=0.345):
    weekly_df = df.copy()
    
    # Convert '% Change' to numeric, handling any non-numeric values
    weekly_df['% Change'] = pd.to_numeric(weekly_df['% Change'], errors='coerce')
    
    # Iterate through each 'Language/Market/Iban' group
    for language in weekly_df['Language/Market/Iban'].unique():
        lang_df = weekly_df[weekly_df['Language/Market/Iban'] == language]
        lang_indices = lang_df.index.tolist()
        
        for i in range(1, len(lang_indices)):
            current_idx = lang_indices[i]
            prev_idx = lang_indices[i-1]
            
            # Check if 'MAU' values are identical and the current row is 'interpolated'
            if (weekly_df.loc[current_idx, 'MAU'] == weekly_df.loc[prev_idx, 'MAU'] and
                weekly_df.loc[current_idx, 'Type'] == 'interpolated' and
                weekly_df.loc[current_idx, '% Change'] == 0):
                
                # Apply the smoothing factor to the 'MAU' value
                prev_mau = weekly_df.loc[prev_idx, 'MAU']
                new_mau = round(prev_mau * (1 - smoothness_factor / 100))
                
                # Update the 'MAU' and '% Change' for the interpolated row
                weekly_df.loc[current_idx, 'MAU'] = new_mau
                weekly_df.loc[current_idx, '% Change'] = -smoothness_factor  # Store the negative smoothing factor
                
    # Format '% Change' to two decimal places
    weekly_df['% Change'] = weekly_df['% Change'].apply(lambda x: f"{x:.2f}" if pd.notnull(x) else '')
    
    return weekly_df



In [554]:
corrected_df = smooth_interpolate_mau(weekly_df, smoothness_factor=-0.345)
corrected_df

,Week Starting,Language/Market/Iban,Type,MAU,% Change,Calendar Week
4,2024-01-29,de,abs,696736,,2024-w05
5,2024-02-05,de,interpolated,698414,0.24,2024-w06
6,2024-02-12,de,interpolated,700097,0.24,2024-w07
7,2024-02-19,de,interpolated,701783,0.24,2024-w08
8,2024-02-26,de,abs,703474,0.24,2024-w09
9,2024-03-04,de,interpolated,705977,0.36,2024-w10
10,2024-03-11,de,interpolated,708488,0.36,2024-w11
11,2024-03-18,de,interpolated,711009,0.36,2024-w12
12,2024-03-25,de,abs,713538,0.36,2024-w13
13,2024-04-01,de,interpolated,716000,0.34,2024-w14


In [556]:
forecast_name = "[WFM-REMAP]_2nd_submision_2025_Budget_fc_[with wfm 2024-2027 extrapolation]_Weekly_[Language_Basis]"
forecast_date = "06/11/2024"

corrected_df['Forecast Name'] = forecast_name
corrected_df['Grouping Type'] = 'Language'
corrected_df['Forecast Date'] = forecast_date
corrected_df['Calendar Week'] = corrected_df['Week Starting'].apply(lambda x: pd.to_datetime(x).strftime('%Y-w%U'))

corrected_df = corrected_df[['Forecast Name',	'Grouping Type', 'Forecast Date', 'Calendar Week', 'Week Starting', 'Language/Market/Iban', 'Type',
           'MAU','% Change']]

corrected_df.head()


,Forecast Name,Grouping Type,Forecast Date,Calendar Week,Week Starting,Language/Market/Iban,Type,MAU,% Change
4,[WFM-REMAP]_2nd_submision_2025_Budget_fc_[with...,Language,06/11/2024,2024-w04,2024-01-29,de,abs,696736,
5,[WFM-REMAP]_2nd_submision_2025_Budget_fc_[with...,Language,06/11/2024,2024-w05,2024-02-05,de,interpolated,698414,0.24
6,[WFM-REMAP]_2nd_submision_2025_Budget_fc_[with...,Language,06/11/2024,2024-w06,2024-02-12,de,interpolated,700097,0.24
7,[WFM-REMAP]_2nd_submision_2025_Budget_fc_[with...,Language,06/11/2024,2024-w07,2024-02-19,de,interpolated,701783,0.24
8,[WFM-REMAP]_2nd_submision_2025_Budget_fc_[with...,Language,06/11/2024,2024-w08,2024-02-26,de,abs,703474,0.24


In [ ]:
# from datetime import datetime, timedelta

# def get_weekly_dates(start_date, end_date):
#     return pd.date_range(start=start_date, end=end_date, freq='W-MON')

# start_date = datetime(2024, 1, 29)  # Last Monday of Jan 2024
# end_date = datetime(2025, 12, 29)  # Last Monday of Dec 2025

# #weekly Mondays
# dates = get_weekly_dates(start_date, end_date)

# # Define static values
# forecast_name = "[WFM-REMAP]_2nd_submision_2025_Budget_fc_[with wfm 2026-2027 extrapolation]_Weekly_[Language_Basis]"
# grouping_type = "Language"
# forecast_date = "06/11/2024"

# final_data = []

# for language, group in sorted_df.groupby("Language"):
#     group = group.reset_index(drop=True)
#     weekly_data = []
#     weekly_types = []

#     for i in range(len(group) - 1):
#         start_mau = group.loc[i, "MAU"]
#         end_mau = group.loc[i + 1, "MAU"]
#         mom_growth = group.loc[i + 1, "MoM Growth (%)"]

#         weekly_growth_rate = (1 + mom_growth / 100) ** (1 / 4) - 1 if pd.notna(mom_growth) else 0  

#         weekly_mau = [start_mau]  # First value of the month
#         weekly_types.append("abs")

#         for _ in range(3): 
#             weekly_mau.append(weekly_mau[-1] * (1 + weekly_growth_rate))
#             weekly_types.append("interpolated")

#         weekly_data.extend(weekly_mau)

#     extra_weeks_needed = len(dates) - len(weekly_data)
#     if extra_weeks_needed > 0:
#         last_mau = weekly_data[-1]
#         last_growth_rate = weekly_growth_rate
#         for _ in range(extra_weeks_needed):
#             last_mau *= (1 + last_growth_rate)
#             weekly_data.append(last_mau)
#             weekly_types.append("interpolated")

#     weekly_df = pd.DataFrame({
#         "Date": dates[:len(weekly_data)],
#         "MAU": [int(m) for m in weekly_data],
#         "Type": weekly_types})

#     weekly_df["Week Num"] = weekly_df["Date"].dt.isocalendar().week
#     weekly_df["Type"] = np.where(weekly_df["Week Num"] % 4 == 1, "abs", "interpolated")
    
#     weekly_df["Calendar Week"] = weekly_df["Date"].dt.strftime('%Y-w%V')
#     # weekly_df["MAU"] = weekly_df["MAU"].astype(int)

#     weekly_pct_change = [np.nan] + [(weekly_data[i] - weekly_data[i-1]) / weekly_data[i-1] * 100 for i in range(1, len(weekly_data))]

#     language_df = pd.DataFrame({
#         "Forecast Name": forecast_name,
#         "Grouping Type": grouping_type,
#         "Forecast Date": forecast_date,
#         "Calendar Week": weekly_df["Calendar Week"],
#         "Week Starting": weekly_df["Date"].dt.strftime('%d/%m/%Y'), 
#         "Language/Market/Iban": language,
#         "Type": weekly_df["Type"].tolist(),
#         "MAU": weekly_df["MAU"].tolist(),  
#         "% Change": [f"{p:.2f}%" if not np.isnan(p) else "" for p in weekly_pct_change]})
    
#     final_data.append(language_df)

# final_df = pd.concat(final_data, ignore_index=True)
# final_df.head(7)

In [ ]:
# # Function to split language values into weekly values
# def split_language_to_weekly(df):
#     weekly_rows = []

#     for _, row in df.iterrows():
#         language = row['Language']
#         language_mau = row['MAU']  # This is the language-specific MAU

#         # Step 1: Generate 4 random values that sum up to the language MAU, ensuring no zero values
#         while True:
#             # Generate 4 random values (non-zero) and make sure they sum to the total language MAU
#             weekly_values = np.random.uniform(0.01, language_mau, 4)  # Ensuring no zero values
#             weekly_values = np.round(weekly_values, 2)  # Round to 2 decimal places

#             if np.sum(weekly_values) <= language_mau:
#                 diff = language_mau - np.sum(weekly_values)
#                 weekly_values[-1] += diff
#                 break

#         # Step 2: Add weekly values to the result
#         for week_num, weekly_value in enumerate(weekly_values, start=1):
#             month_name = row['Calendar Week'][:4]
#             year = '20' + row['Calendar Week'][5:7]
#             new_row = {
#                 'Calendar Week': f'{year}-{month_name}-{week_num:02d}', 
#                 'Country': row['Country'],
#                 'Language': language,
#                 'MAU': np.round(weekly_value, 2)  # Round to 2 decimal places
#             }
#             weekly_rows.append(new_row)
    
#     # Convert the weekly rows into a new DataFrame
#     weekly_df = pd.DataFrame(weekly_rows)
#     return weekly_df



In [ ]:
# weekly_df = split_language_to_weekly(mapped_df)
# weekly_df.head()

In [ ]:
# weekly_df['Calendar Week'] = pd.to_datetime(weekly_df['Calendar Week'], errors='coerce')

# weekly_df['Month_Week_Label'] = weekly_df['Calendar Week'].dt.to_period('M').astype(str).str.replace('-', '-w')

# # Group by Month-Week label, Country, and Language, summing MAU values
# grouped_df = weekly_df.groupby(['Month_Week_Label', 'Country', 'Language'], as_index=False)['MAU'].sum()

# # Compute '% Change' for each Country-Language group per month
# grouped_df['% Change'] = grouped_df.groupby(['Country', 'Language'])['MAU'].pct_change() * 100
# grouped_df['% Change'] = grouped_df['% Change'].fillna(0).apply(lambda x: f"{x:.2f}%")

# grouped_df.rename(columns={'Language': 'Language/Market/Iban'}, inplace=True)

# grouped_df['MAU'] = grouped_df['MAU'].apply(lambda x: f"{x:,.0f}")
# grouped_df = grouped_df.sort_values(by=['Country', 'Language/Market/Iban', 'Month_Week_Label'], ascending=[True, True, True])
# grouped_df.head()


In [ ]:
# grouped_df.tail()

In [ ]:
# from prophet import Prophet 
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# import warnings
# warnings.filterwarnings("ignore")
# import seaborn as sns

# def mean_absolute_perc_error(y_true, y_pred):
#     """ Calcualte MAPE"""
#     y_true, y_pred = np.array(y_true) - np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred)/y_true)) *100


# sns.set_style("whitegrid")

# plt.figure(figsize=(11, 5))
# sns.lineplot(x="Month_Week_Label", y="MAU", data=grouped_df, marker="o", color=sns.color_palette()[0])

# plt.title("Monthly active users")
# plt.xlabel("Month_Week_Label")
# plt.ylabel("MAU")
# plt.xticks(rotation=45)
# plt.show()


In [ ]:
# import pandas as pd
# import numpy as np
# from prophet import Prophet
# from datetime import datetime, timedelta

# def prepare_data_for_prophet(df):
#     """Prepare the data for Prophet while preserving weekly patterns"""
#     prophet_df = df.copy()
    
#     prophet_df['MAU'] = prophet_df['MAU'].str.replace(',', '').astype(float)
#     prophet_df['week'] = prophet_df['Month_Week_Label'].str.extract('w(\d+)').astype(int)
#     prophet_df['year'] = prophet_df['Month_Week_Label'].str.split('-').str[0].astype(int)
    
#     prophet_df['ds'] = pd.to_datetime(prophet_df.apply(
#         lambda x: f"{x['year']}-01-01", axis=1)) + pd.to_timedelta((prophet_df['week'] - 1) * 7, unit='D')
    
#     prophet_df['y'] = prophet_df['MAU']
#     return prophet_df

# def analyze_historical_patterns(df):
#     """Analyze historical weekly patterns to inform forecasting"""
#     df['pct_change'] = df['MAU'].str.replace(',', '').astype(float).pct_change() * 100
#     weekly_patterns = df.groupby('Month_Week_Label')['pct_change'].mean()
#     return weekly_patterns

# def generate_smooth_forecast(grouped_df):
#     """Generate forecasts with realistic variations based on historical patterns"""
#     all_forecasts = []
    
#     for (country, language), group_data in grouped_df.groupby(['Country', 'Language/Market/Iban']):
#         try:
#             historical_patterns = analyze_historical_patterns(group_data)
            
#             # Prepare data
#             df = prepare_data_for_prophet(group_data)
#             last_value = df['y'].iloc[-1]
            
#             model = Prophet(
#                 growth='linear',
#                 changepoint_prior_scale=0.005,  
#                 seasonality_prior_scale=0.1,   
#                 yearly_seasonality=True,
#                 weekly_seasonality=True,
#                 daily_seasonality=False)
            
#             model.add_seasonality(
#                 name='twelve_week',
#                 period=12 * 7,
#                 fourier_order=5)
            
#             # Fit model
#             model.fit(df)
            
#             # Generate future dates
#             future_weeks = []
#             for year in [2026, 2027]:
#                 for week in range(1, 13):  # Weeks 1-12
#                     future_weeks.append(f"{year}-w{week:02d}")
            
#             future_dates = pd.DataFrame({
#                 'Month_Week_Label': future_weeks,
#                 'ds': [pd.to_datetime(f"{w.split('-')[0]}-01-01") + 
#                       pd.Timedelta(weeks=int(w.split('w')[1])-1) for w in future_weeks]})
            
#             forecast = model.predict(future_dates)
#             results = pd.DataFrame({
#                 'Month_Week_Label': future_weeks,
#                 'Country': country,
#                 'Language/Market/Iban': language,
#                 'MAU': forecast['yhat'].round(0)})
            
#             current_value = last_value
#             for idx in results.index:
#                 week_label = results.loc[idx, 'Month_Week_Label']
#                 week_num = int(week_label.split('w')[1])
                
#                 hist_pattern = historical_patterns.get(week_label, 0)
#                 growth_rate = np.clip(hist_pattern / 100, -0.01, 0.01)                
#                 variation = np.random.uniform(-0.002, 0.002)  # Small random variation
#                 growth_rate = np.clip(growth_rate + variation, -0.01, 0.01)
                
#                 current_value = current_value * (1 + growth_rate)
#                 results.loc[idx, 'MAU'] = current_value
            
#             # Calculate actual % change
#             results['MAU'] = results['MAU'].round(0)
#             results['% Change'] = (results['MAU'].pct_change().fillna(0) * 100)
            
#             results['MAU'] = results['MAU'].apply(lambda x: f"{int(x):,}")
#             results['% Change'] = results['% Change'].apply(lambda x: f"{x:.2f}%")
            
#             all_forecasts.append(results)
            
#         except Exception as e:
#             print(f"Error forecasting for {country}-{language}: {str(e)}")
#             continue
    
#     if all_forecasts:
#         final_forecasts = pd.concat(all_forecasts, ignore_index=True)
#         return final_forecasts.sort_values(['Country', 'Language/Market/Iban', 'Month_Week_Label']).reset_index(drop=True)
    
#     return pd.DataFrame(columns=['Month_Week_Label', 'Country', 'Language/Market/Iban', 'MAU', '% Change'])

# def run_forecast(grouped_df):
#     """Run the forecast with proper error handling"""
#     try:
#         return generate_smooth_forecast(grouped_df)
#     except Exception as e:
#         print(f"Error in forecasting: {str(e)}")
#         return None

In [ ]:
# # Generate forecasts
# forecasts = run_forecast(grouped_df)
# forecasts.tail(10)

In [ ]:
# # concatenate both df: interpolated one + extraplated one
# df = pd.concat([grouped_df, forecasts], ignore_index=True)
# df.tail()

In [ ]:
# df.head()

### Export expanded_df to copied_marketing gsheet we created first thing

In [557]:
corrected_df['Week Starting'] = pd.to_datetime(corrected_df['Week Starting']).dt.strftime('%Y-%m-%d')

# Now, export to Google Sheets
gsheet_export_params = dict(
    df = corrected_df, 
    gsheet_id = parameters["sheet_id"],
    gsheet_tab_name = 'Final',
    include_df_headers = True,
    tab_colour = (0.0, 0.0, 0.0))

fc.export_df_to_google_sheet(**gsheet_export_params)
